In [ ]:
import json
import os

import numpy as np
import pandas as pd
import tqdm
from itables import init_notebook_mode

import src.graphics as graphics

init_notebook_mode(all_interactive=True)

In [ ]:
REPORTS_CSV = os.path.abspath("./resources/megadetector_coverage/MegaDetector_0_003.csv")
REPORTS_FOLDER = os.path.abspath("./reports/fase_2")

# subset_option = "all"
# subset_option = "train"
# subset_option = "validation"
subset_option = "test"

In [ ]:
report = pd.read_csv(REPORTS_CSV, sep=";")

model = os.path.basename(REPORTS_CSV).replace(".csv", "")
threshold = model.replace("MegaDetector_", "")

In [ ]:
if subset_option != "all":
    report = report[report["subset"] == subset_option]

In [ ]:
real_positives = report["binary_label"].sum()
covered_positives = report[
    (report["binary_label"] == 1) & (report["detector_label"] == 1)
    ].shape[0]

percentage_covered_positives = (
    (covered_positives / real_positives) * 100 if real_positives > 0 else 0
)
print(f"Percentage of covered positives: {percentage_covered_positives:.2f}%")
print(f"There is {real_positives} real positives")
print(f"There is {covered_positives} covered positives")

In [ ]:
TP = ((report["binary_label"] == 1) & (report["detector_label"] == 1)).sum()
FP = ((report["binary_label"] == 0) & (report["detector_label"] == 1)).sum()
TN = ((report["binary_label"] == 0) & (report["detector_label"] == 0)).sum()
FN = ((report["binary_label"] == 1) & (report["detector_label"] == 0)).sum()

conf_matrix = np.array([[TN, FN], [FP, TP]])

print(conf_matrix)

In [ ]:
fig = graphics.create_heatmap_chart(
    conf_matrix=conf_matrix,
    x_data=["Empty", "Animal"],
    y_data=["Empty", "Animal"],
    conf_matrix_text=[[str(value) for value in row] for row in conf_matrix],
    title="MegaDetector Threshold = " + threshold.replace("_", ".") + " (Subset: " + subset_option + ")",
    x_title="MegaDetector",
    y_title="Dataset")
fig.show()

os.makedirs(REPORTS_FOLDER, exist_ok=True)
fig.write_image(os.path.join(REPORTS_FOLDER, f"confusion_matrix_{model}_{subset_option}.png"))